In [10]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
from io import StringIO
import time

In [12]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [14]:
data = requests.get(standings_url)

In [16]:
soup = bs(data.text)

In [17]:
standings_table = soup.select("table.stats_table")[0]

In [18]:
links = standings_table.find_all("a")

In [19]:
links = [link.get("href") for link in links]

In [20]:
links = [link for link in links if "/squads/" in link]

In [21]:
team_urls = [f"https://fbref.com{link}" for link in links]

In [22]:
team_url = team_urls[0]
data = requests.get(team_url)

In [24]:
matches = pd.read_html(StringIO(data.text), match="Scores & Fixtures")

In [25]:
soup = bs(data.text)